# Dot dispersion of decoded transcripts

This notebook can be used to measure the general spread of spot distances used for decoding a transcript. This information may be useful when comparing different probe design methodoligies.

In [ ]:
import tifffile as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'

In [ ]:
def used_dots_decoded_genes(df_locs_2d, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d.x,
        y=df_locs_2d.y,
        mode='markers',
        marker_symbol='circle',
        marker=dict(
            #maxdisplayed=1000,
            size=5, color = "blue",
            ),
        name = "Used"
        ))
    
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='circle',
            marker=dict(
                #maxdisplayed=1000,
                size=5, color="green"
                ),
            name = "Decoded On"
            ))
        
        fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01))
    
        
    fig.show()
    

In [ ]:
def keep_dots_in_cells(mask, dot_locations):
    """
    A function to remove any dots outside of mask
    Parameter
    ---------
    mask = cellpose generated mask path
    dot_locations = dot_locations path
    """
    
    #read in data
    locations = pd.read_csv(dot_locations)
    #cellpose mask outputs (c,y,x)
    img = tf.imread(mask)
    #get x and y coordinates
    locations_xy = locations[["x","y"]].values.astype(int)
    dot_info = []
    #keep dots only in cells
    for i in range(len(locations)):
        x = locations_xy[i][0]
        y = locations_xy[i][1]
        if img[y,x] == 0:
            continue
        else:
            cell = img[y,x]
            dot_info.append([i,cell])
            
    dot_info = np.array(dot_info)
    
    #keep rows that have cells
    dots_in_cells = locations.loc[dot_info[:,0]]
    
    #add cell info
    dots_in_cells["cell number"] = dot_info[:,1]
    
    return dots_in_cells

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

from sklearn.neighbors import KDTree


def dot_displacement_from_gene(ref,dots, rounds=3, distance_cutoff=2):
    
    """
    A function to measure the distance between decoded transcripts and the dots that make up that decoded 
    transcript.
    
    Parameters
    -----------
    ref: decoded transcript locations with fakes filtered and ambiguity=0
    dots: locations of dots used in decoding mapped to cell masks
    distance_cutoff: the maximum search radius used in decoding
    """
    
    #initialize neighbor
    neigh = NearestNeighbors(n_neighbors=1, metric="euclidean", n_jobs=1)
    
    barcoding_round = []
    #separate locations by barcoding round
    hyb_rounds = np.arange(0, len(dots["hyb"].unique()),1)
    hybs = len(dots["hyb"].unique())
    temp = []
    for h in hyb_rounds:
        if h == hyb_rounds[len(hyb_rounds)-1]:
            barcode = dots[dots["hyb"] == h]
            temp.append(barcode)
            comp_round = pd.concat(temp)
            barcoding_round.append(comp_round) 
        elif (h % (hybs/rounds) != 0) or (h == 0):
            barcode = dots[dots["hyb"] == h]
            temp.append(barcode)
        else:
            comp_round = pd.concat(temp)
            barcoding_round.append(comp_round)
            temp = []
            barcode = dots[dots["hyb"] == h]
            temp.append(barcode)

    distance_per_round = []
    index_list = []
    for seed in barcoding_round:
        seed = seed.reset_index(drop=True)
        #initialize KDTree
        kdt = KDTree(seed[["x","y"]].values, leaf_size=40, metric='euclidean')
        #get nearest neighbor
        distance, index = kdt.query(ref[["x","y"]].values, k=1, return_distance=True)
        #get distances below cutoff
        distance = np.compress(distance.ravel()<distance_cutoff, distance.ravel())
        #add distance and index to list
        distance_per_round.append(distance)
        index_list.append(seed[["x","y"]].values[index][:,0])
    
    return distance_per_round, index_list, hyb_rounds, hybs

In [ ]:
#read in one of the gene mapped locations file
locations = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr5/Channel_All/Pos_0/diff_1_minseed_3_z_2_finalgenes.csv", index_col=0)
#get dots used
dots_used_locations = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/dots_detected/Channel_All/Pos0/locations_z_2.csv", index_col=0)
#remove fakes
fakes = locations[locations["genes"].str.startswith("fake")]
dots_used_locations = dots_used_locations[dots_used_locations.hyb < 8]

In [ ]:
# img_2d = tf.imread(f"/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/chromatic_aberration/MMStack_Pos0.ome.tif")[0][2]
# used_dots_decoded_genes(dots_used_locations, locations, img_2d, add_trace = True, zmax=1000)

In [ ]:
#reset index
dots_used = dots_used_locations.reset_index(drop=True)
#get dot displacement
distances, index, a, b = dot_displacement_from_gene(locations, dots_used, rounds=4, distance_cutoff=2)

In [ ]:
distances

In [ ]:
_min = min(min(distances[0]*100), min(distances[1]*100), min(distances[2]*100), min(distances[3]*100))
_max = max(max(distances[0]*100), max(distances[1]*100), max(distances[2]*100), max(distances[3]*100))

color = ["red","blue","green", "orange"]
for i in range(4):
    plt.hist(distances[i]*100, bins=20, range=(_min,_max), alpha=0.2, color = color[i], label=f"Round {i+1}")
plt.legend()
sns.despine()
plt.xlabel("Distance (nm)")
plt.ylabel("Counts")
plt.show()

In [ ]:
np.mean([np.mean(dist) for dist in distances])

In [ ]:
#read in one of the gene mapped locations file
locations = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p15_diff0_newdecoder2/Channel_1/Pos_0/diff_0_minseed_3_z_0_finalgenes.csv", index_col=0)
#get dots used
dots_used_locations = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/dots_detected/Channel_1_processed/genes_in_cells/Pos0/locations_z_0.csv", index_col=0)
#remove fakes
fakes = locations[locations["genes"].str.startswith("fake")]
dots_used_locations = dots_used_locations[dots_used_locations.hyb < 24]

In [ ]:
#reset index
dots_used = dots_used_locations.reset_index(drop=True)
#get dot displacement
distances, index, a, b = dot_displacement_from_gene(locations, dots_used, rounds=4, distance_cutoff=2)

In [ ]:
distances

In [ ]:
_min = min(min(distances[0]*100), min(distances[1]*100), min(distances[2]*100), min(distances[3]*100))
_max = max(max(distances[0]*100), max(distances[1]*100), max(distances[2]*100), max(distances[3]*100))

color = ["red","blue","green", "orange"]
for i in range(4):
    plt.hist(distances[i]*100, bins=20, range=(_min,_max), alpha=0.2, color = color[i], label=f"Round {i+1}")
plt.legend()
sns.despine()
plt.xlabel("Distance (nm)")
plt.ylabel("Counts")
plt.show()

In [ ]:
np.mean([np.mean(dist) for dist in distances])